![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [113]:
# Start your code here!
import pandas as pd

# Reading original data

In [114]:
ds_jobs = pd.read_csv("customer_train.csv")

Let's see how much memory the Dataframe is using

In [115]:
ds_jobs.shape

(19158, 14)

In [116]:
ds_jobs.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [117]:
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

# Dataframe cleaning

In [118]:
ds_jobs_clean = ds_jobs.copy()

## Integer columns

In [119]:
# Columns containing integers must be stored as 32-bit integers (int32).

# Filter columns with integer data type
integer_columns = [col for col, dtype in ds_jobs_clean.dtypes.items() if dtype == "int64"]
print(f"Columns with data type int64 are: \n{integer_columns}")

for col in integer_columns:
    ds_jobs_clean[col] = ds_jobs_clean[col].astype("int32")

print(ds_jobs_clean[integer_columns].dtypes)

Columns with data type int64 are: 
['student_id', 'training_hours', 'job_change']
student_id        int32
training_hours    int32
job_change        int32
dtype: object


## Float columns

In [120]:
#Columns containing floats must be stored as 16-bit floats (float16).

# Filter columns with float data type
float_columns = [col for col, dtype in ds_jobs_clean.dtypes.items() if dtype == "float64"]
print(f"Columns with data type float64 are: \n{float_columns}")

for col in float_columns:
    ds_jobs_clean[col] = ds_jobs_clean[col].astype("float16")
    
print(ds_jobs_clean[float_columns].dtypes)

Columns with data type float64 are: 
['city_development_index']
city_development_index    float16
dtype: object


## Categorical columns

### Nominals

In [121]:
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.775879,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624023,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789062,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767090,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [122]:
# Columns containing nominal categorical data must be stored as the category data type.
nominal_columns = ["city", "gender", "major_discipline", "company_type"]
for c in nominal_columns:
    ds_jobs_clean[c] = ds_jobs_clean[c].astype("category")
    
print(ds_jobs_clean[nominal_columns].dtypes)

city                category
gender              category
major_discipline    category
company_type        category
dtype: object


In [123]:
# Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
ordinal_columns = ["relevant_experience", "enrolled_university", "education_level",
                   "experience", "company_size", "last_new_job"]
print(ds_jobs_clean[ordinal_columns].dtypes)

relevant_experience    object
enrolled_university    object
education_level        object
experience             object
company_size           object
last_new_job           object
dtype: object


In [125]:
for c in ordinal_columns:
    print(f"Unique values from column {c}")
    print(ds_jobs_clean[c].unique())

Unique values from column relevant_experience
['Has relevant experience' 'No relevant experience']
Unique values from column enrolled_university
['no_enrollment' 'Full time course' nan 'Part time course']
Unique values from column education_level
['Graduate' 'Masters' 'High School' nan 'Phd' 'Primary School']
Unique values from column experience
['>20' '15' '5' '<1' '11' '13' '7' '17' '2' '16' '1' '4' '10' '14' '18'
 '19' '12' '3' '6' '9' '8' '20' nan]
Unique values from column company_size
[nan '50-99' '<10' '10000+' '5000-9999' '1000-4999' '10-49' '100-499'
 '500-999']
Unique values from column last_new_job
['1' '>4' 'never' '4' '3' '2' nan]


In [126]:
from pandas.api.types import CategoricalDtype

ordinal_cats = [
    CategoricalDtype(categories=['No relevant experience', 
                                 'Has relevant experience'],
                     ordered=True),
    
    CategoricalDtype(categories=['Has relevant experience','no_enrollment',
                                'Part time course','Full time course'],
                     ordered=True),
    
    CategoricalDtype(categories=['Primary School', 'High School',
                                 'Graduate','Masters','Phd' ],
                     ordered=True),
    
    CategoricalDtype(categories=['<1','1','2','3','4','5','6','7','8','9','10','11',
                                 '12','13','14', '15', '16','17','18','19','20','>20'],
                     ordered=True),
    
    CategoricalDtype(categories=['<10', '10-49', '50-99','100-499',
                                 '500-999', '1000-4999', '5000-9999', '10000+'],
                     ordered=True),
    
    CategoricalDtype(categories=['never', '1', '2', '3', '4', '>4'],
                     ordered=True),
]

In [127]:
for c, cat in zip(ordinal_columns, ordinal_cats):
    ds_jobs_clean[c] = ds_jobs_clean[c].astype(cat)
    
print(ds_jobs_clean[ordinal_columns].dtypes)

relevant_experience    category
enrolled_university    category
education_level        category
experience             category
company_size           category
last_new_job           category
dtype: object


# Filter students

Let's check if ordinal columns works

In [128]:
ds_jobs_clean[ds_jobs_clean["experience"] >= "10"].sample(10)[["student_id", "experience"]]

,student_id,experience
8284,13465,11
10588,806,10
10421,29377,14
2924,11418,16
10621,17014,10
12324,3715,>20
1516,7713,18
11397,26959,12
5257,25080,>20
15514,16134,11


In [129]:
ds_jobs_clean[ds_jobs_clean["company_size"] >= "1000-4999"].sample(10)[["student_id", "company_size"]]

,student_id,company_size
18735,23250,10000+
2720,3909,1000-4999
3766,29932,10000+
4925,23201,10000+
18812,15625,5000-9999
15405,22600,1000-4999
12302,17010,1000-4999
12866,8460,1000-4999
11823,10886,5000-9999
18113,20752,10000+


In [130]:
# The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

ds_jobs_clean = ds_jobs_clean[ (ds_jobs_clean["experience"] >= "10") &
                               (ds_jobs_clean["company_size"] >= "1000-4999") ]

In [131]:
ds_jobs_clean.sample(5)[["experience", "company_size"]]

,experience,company_size
18574,11,10000+
14042,>20,10000+
15717,18,5000-9999
34,12,5000-9999
17879,17,5000-9999


We have filtered the employees successfully!

# Compare Dataframes memory use

## Original

In [132]:
ds_jobs.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [133]:
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [134]:
ds_jobs_clean.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   int32   
dtypes: category(10), float1

In [135]:
ds_jobs_clean.memory_usage()

Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2325
enrolled_university        2405
education_level            2413
major_discipline           2421
experience                 2933
company_size               2565
company_type               2421
last_new_job               2421
training_hours             8804
job_change                 8804
dtype: int64

Original dataframe uses 11.8 MB since our cleaned dataframe uses 86.3 KB (0.0863 MB).
Getting a reduction rate of $\frac{11.8 \ MB}{0.0863 \ MB} = 136.73$, it means we have compressed our file 136 times!!.